In [30]:
def create_single_scenic_environment(iprocess, level, monitor=None):
    """Creates scenic gfootball environment."""
    from scenic.simulators.gfootball.rl_interface import GFScenicEnv
    import os
    from scenic.simulators.gfootball.utilities.scenic_helper import buildScenario

    #scenario_file = f"{os.getcwd()}/_scenarios/exp/pass_n_shoot.scenic"
    scenario_file = level
    print("Scenic Environment: ", scenario_file)

    gf_env_settings = {
        "stacked": True,
        "rewards": "scoring",
        "representation": 'extracted',
        "players": [f"agent:left_players=1"],
        "real_time": False,
        "action_set": "default",
        "dump_full_episodes": False,
        "dump_scores": False,
        "write_video": False,
        "tracesdir": "dummy",
        "write_full_episode_dumps": False,
        "write_goal_dumps": False,
        "render": False
    }

    scenario = buildScenario(scenario_file)
    env = GFScenicEnv(initial_scenario=scenario, gf_env_settings=gf_env_settings, rank=iprocess)
    if monitor is not None:
        env = monitor.Monitor(env, logger.get_dir() and os.path.join(logger.get_dir(), str(iprocess)), info_keywords=("score_reward",))
    return env

In [32]:
env = create_single_scenic_environment(0, "../_scenarios/academy/rts.scenic")

Scenic Environment:  ../_scenarios/academy/rts.scenic


In [33]:
from collections import namedtuple
Trajectory = namedtuple('Trajectory', ["obs_arr", "gt_arr", "act_arr", "rew_arr", "info_arr"])

def collect_trajectory_with_ground_truth(env, num_traj=100, policy=None):
    
    trajs = []
    
    for _ in range(num_traj):
        
        obs_arr, gt_arr, act_arr, rew_arr, info_arr = [], [], [], [], []
        
        obs = env.reset()
        gt = env.simulation.last_raw_obs
        
        obs_arr.append(obs)
        gt_arr.append(gt)
        
        done = False
        
        while not done:
            
            act = env.action_space.sample()
            obs, r, done, info = env.step(act)
            gt = env.simulation.last_raw_obs
            
            act_arr.append(act)
            obs_arr.append(obs)
            gt_arr.append(gt)
            rew_arr.append(r)
            info_arr.append(info)
        
        
        traj = Trajectory(obs_arr=obs_arr, gt_arr=gt_arr, act_arr=act_arr, rew_arr=rew_arr, info_arr=info_arr)
        trajs.append(traj)
        
    
    return trajs



In [35]:
import pickle

random_trajs_5 = collect_trajectory_with_ground_truth(env, num_traj=5)
pickle.dump(random_trajs_5, open("rts_rand_traj_5", "wb"))


KeyboardInterrupt: 

In [25]:
import pickle

#random_trajs_100 = collect_trajectory_with_ground_truth(env, num_traj=100)
pickle.dump(random_trajs_100, open("rts_rand_traj_100", "wb"))
